In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install fastapi

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
movie=pd.read_csv(r"C:\Users\Sanvi Arora\OneDrive\Desktop\ratings.csv")
movie.head()
movie=movie.drop('timestamp',axis=1)

movie_ratings=pd.DataFrame()
movie_ratings['movieId']=movie['movieId']
movie_ratings['rating']=movie['rating']

user_ratings=pd.DataFrame()
user_ratings['userId']=movie['userId']
user_ratings['rating']=movie['rating']

count_ratings_users=pd.DataFrame(user_ratings.groupby('userId',as_index=False).count())
count_ratings_users=count_ratings_users.sort_values(by='rating',ascending=False)
count_ratings_users.rename(columns = {'rating':'Count'}, inplace = True)
count_ratings_users=count_ratings_users[count_ratings_users['Count']>=5]

df0=count_ratings_users.merge(movie,on='userId',how='inner')

count_ratings_movie=pd.DataFrame(movie_ratings.groupby('movieId',as_index=False).count())
count_ratings_movie=count_ratings_movie.sort_values(by='rating',ascending=False)
count_ratings_movie.rename(columns = {'rating':'Count'}, inplace = True)
count_ratings_movie=count_ratings_movie[count_ratings_movie['Count']>=75]
movie=count_ratings_movie.merge(df0,on='movieId', how='inner')
movie=movie.drop(columns=['Count_x','Count_y'],axis=1)

movie['movieId']=movie['movieId'].rank(method='dense').astype(int)
movie['userId']=movie['userId'].rank(method='dense').astype(int)



In [3]:
from fastapi import FastAPI
app=FastAPI()
import joblib
user_item_matrix_norm = pd.read_csv(r"C:\Users\Sanvi Arora\Downloads\user_item_matrix_norm (2).csv")
item_similarity = pd.read_csv(r"C:\Users\Sanvi Arora\Downloads\item_similarity (2).csv")

# Define a route for prediction
def predict_rating(userId, movieId, max_neighbor=2): 
    movies = item_similarity.sort_values(by=movieId, ascending=False).index[1:]
    scores = item_similarity.sort_values(by=movieId, ascending=False).loc[:, movieId].tolist()[1:]
    movies_arr = np.array([x for x in movies])
    sim_arr = np.array([x for x in scores])
    # select only the movie that has already rated by given user
    filtering = user_item_matrix_norm[userId].loc[movies_arr] != 0
    # calculate the predicted score
    s = np.dot(sim_arr[filtering][:max_neighbor], user_item_matrix_norm[userId].loc[movies_arr[filtering][:max_neighbor]])/ np.sum(sim_arr[filtering][:max_neighbor])
    return s
@app.get('/predict/{userId}/{n}')
def get_recommendation(userId, n=5):
    predicted_rating = np.array([])
    for movie_ in user_item_matrix_norm.index:
        predicted_rating = np.append(predicted_rating, predict_rating(userId, movie_))
    # don't recommend something that user has already rated
    temp = pd.DataFrame({'predicted_rating':predicted_rating, 'movie':user_item_matrix_norm.index})
    filtering = (user_item_matrix_norm[userId] == 0.0)
    temp = temp.loc[filtering.values].sort_values(by='predicted_rating', ascending=False)
    # recommend n movies to user
    return temp.movie[:n].tolist()